# Cause Matching 

## Data Preprocessing and Analysis

In [ ]:
import pandas as pd
data = pd.read_csv('Study Title REV with Sample.csv')

In [ ]:
data

In [ ]:
Titles = data['Public Title of Study']
Causes = data['List of Causes'].dropna()
data = Titles + Causes

In [ ]:
len(Titles)
len(Causes)

Hence we have  255 causes and 330 titles to search on.
Now, we will filter the titles one by one to get to know the keywords of a title and thier properties.

Now, to preprocess the Titles we will remove the mostl commonly words (stopwords) out of it so that it doesnt affect out models accuracy. Also, we will remove the underscores, puctuations, special characters, escape sequence characters.

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(sentence):
    """
    It will take the word and covert it to its meanigfull format
    finally  final finale finalized ---> Final
    """
    sentence =  " ".join([lemmatizer.lemmatize(word)for word in sentence.split()])
    return sentence

In [ ]:
import re
from gensim.parsing.preprocessing import remove_stopwords
def preprocessing(sentence):
   
   ######################### Preprocessing for new_data ##########################e
    #Converting to smaller case
    sentence = sentence.lower()
    #Stop Words Removal
    sentence = remove_stopwords(sentence)
    #line break removal
    sentence = re.sub(r"\r?\\n"," ", sentence)
    #remove special characters
    sentence = re.sub(r'\W+', ' ', sentence)
    #remove numbers
    sentence = sentence.replace('\d+', '')
    sentence = re.sub(r'\b\d+\b', ' ', sentence)
    #remove punctuation
    sentence = sentence.replace('[^\w\s]','')
    #remove underscore
    sentence = sentence.replace('_', ' ')
    #stemming documents(removing ing, ly, s)
    #remove stop words and finally stem
    sentence = lemmatize(sentence)

    return sentence

In [ ]:
preprocessing(" #Prajwal is a good boy.\n. his fav number is 4")

Applying the desired processes on the titles

In [ ]:
Titles = [preprocessing(i) for i in Titles if type(i) is str]

Applying the same preprocessing on CAuses

In [ ]:
Causes = [preprocessing(i) for i in Causes if type(i) is str]

In [ ]:
len(Causes)

# Similarity model Building

### 1. Approaching with jackkard similarity

Logic
 - A title with similar words as that in the cause must be more related to the cause. Hence, since the word count in intersection matters we will be going for Jackkard similarity

Approach - 
- Take a title and a cause. compare the words that are closer to (similar to)the words in the cause. and count the Jackkard similariy between them.

In [ ]:
from nltk.corpus import wordnet
def get_similars(word):
    similars = []
    similars_processed = []
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            similars.append( preprocessing(i.name()))
    
    return similars

In [ ]:
get_similars("smart")

In [ ]:
def get_similar_from_string(sentence:str)->str:
    similars = ''
    for i in sentence.split(' '):
        similars += ' '.join(get_similars(i))
    return preprocessing(similars)
        

In [ ]:
get_similar_from_string("smart boy")

Function to calculate Jackkard similarity

In [ ]:
# x = Cause and Y:Title
def jaccard_similarity(x,y):
  """ returns the jaccard similarity between two lists """
  cause_similars = get_similar_from_string(x)
  intersection_cardinality = len(set.intersection(*[set(cause_similars), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

In [ ]:
jaccard_similarity("bollywood", "Tv actor shit timpass villain story")

# def main

Creating a Datframe to store the desired objects.

In [151]:
# Df = pd.DataFrame(columns= ['Titles', 'Buffer'])
for title in Titles:
    # Df['Titles'].append(title)
    for id, cause in enumerate(Causes):
        # Df[f'Cause {id+1}'] = Df['Buffer']
        # Df[f'Cause {id+1}'].append(cause)
        similarity = 100*jaccard_similarity(cause, title)
        
        if similarity > 100:
            print(similarity)
            print("Titles", title)
            print("Cause", cause)
        # Df[f'Similarity {id+1}'].append(cause)
        
      
        